In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

In [3]:
import os
import torch

# we use AutoModelForSequenceClassification instead of AutoModel as
# the former has a classification head on top of the pretrained model, and
# is easier to train. We just need to specify the number of classes.
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import Trainer, TrainingArguments

#system call
os.system("")

# importing local module
import config
import utils

0

In [4]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'the available device is: {device}')

the available device is: cuda


In [5]:
num_labels = 6
model = (AutoModelForSequenceClassification.from_pretrained(config.model_ckpt, num_labels=num_labels).to(device))

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'pre_classifier.we

In [6]:
from huggingface_hub import interpreter_login
interpreter_login(new_session=False, write_permission=True)

User is already logged in.


In [7]:
emotions = load_dataset('emotion')
emotions.set_format('torch', columns=['text', 'label'])
print(emotions)

No config specified, defaulting to: emotion/split
Found cached dataset emotion (/home/karvsmech/.cache/huggingface/datasets/emotion/split/1.0.0/cca5efe2dfeb58c1d098e0f9eeb200e9927d889b5a03c67097275dfb5fe463bd)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})


In [8]:
emotions_encoded = emotions.map(utils.tokenize, batched=True, batch_size=None)
print(emotions_encoded)

Loading cached processed dataset at /home/karvsmech/.cache/huggingface/datasets/emotion/split/1.0.0/cca5efe2dfeb58c1d098e0f9eeb200e9927d889b5a03c67097275dfb5fe463bd/cache-400b97e1c34e27fd.arrow
Loading cached processed dataset at /home/karvsmech/.cache/huggingface/datasets/emotion/split/1.0.0/cca5efe2dfeb58c1d098e0f9eeb200e9927d889b5a03c67097275dfb5fe463bd/cache-f3ba5e649a299c07.arrow
Loading cached processed dataset at /home/karvsmech/.cache/huggingface/datasets/emotion/split/1.0.0/cca5efe2dfeb58c1d098e0f9eeb200e9927d889b5a03c67097275dfb5fe463bd/cache-9c94bcae10acba70.arrow


DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
})


In [9]:
batch_size = 64
logging_steps = len(emotions_encoded['train'])//64
model_name = f'{config.model_ckpt}-finetuned-emotion'
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=2,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy='epoch',
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  push_to_hub=True,
                                  log_level='error')

In [11]:
trainer = Trainer(model=model, args=training_args,
        compute_metrics=utils.compute_metrics,
        train_dataset=emotions_encoded['train'],
        eval_dataset=emotions_encoded['validation'],
        tokenizer=utils.tokenizer
        )
trainer.train()

/home/karvsmech/PycharmProjects/Pytorch_projects/hugging_face/distilbert-base-uncased-finetuned-emotion is already a clone of https://huggingface.co/karvsmech/distilbert-base-uncased-finetuned-emotion. Make sure you pull the latest changes with `repo.git_pull()`.
/home/karvsmech/PycharmProjects/Pytorch_projects/venv/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss


TrainOutput(global_step=500, training_loss=0.13987269973754882, metrics={'train_runtime': 149.2545, 'train_samples_per_second': 214.399, 'train_steps_per_second': 3.35, 'total_flos': 720342861696000.0, 'train_loss': 0.13987269973754882, 'epoch': 2.0})